# Before you use this template

This template is just a recommended template for project Report. It only considers the general type of research in our paper pool. Feel free to edit it to better fit your project. You will iteratively update the same notebook submission for your draft and the final submission. Please check the project rubriks to get a sense of what is expected in the template.

---

# FAQ and Attentions
* Copy and move this template to your Google Drive. Name your notebook by your team ID (upper-left corner). Don't eidt this original file.
* This template covers most questions we want to ask about your reproduction experiment. You don't need to exactly follow the template, however, you should address the questions. Please feel free to customize your report accordingly.
* any report must have run-able codes and necessary annotations (in text and code comments).
* The notebook is like a demo and only uses small-size data (a subset of original data or processed data), the entire runtime of the notebook including data reading, data process, model training, printing, figure plotting, etc,
must be within 8 min, otherwise, you may get penalty on the grade.
  * If the raw dataset is too large to be loaded  you can select a subset of data and pre-process the data, then, upload the subset or processed data to Google Drive and load them in this notebook.
  * If the whole training is too long to run, you can only set the number of training epoch to a small number, e.g., 3, just show that the training is runable.
  * For results model validation, you can train the model outside this notebook in advance, then, load pretrained model and use it for validation (display the figures, print the metrics).
* The post-process is important! For post-process of the results,please use plots/figures. The code to summarize results and plot figures may be tedious, however, it won't be waste of time since these figures can be used for presentation. While plotting in code, the figures should have titles or captions if necessary (e.g., title your figure with "Figure 1. xxxx")
* There is not page limit to your notebook report, you can also use separate notebooks for the report, just make sure your grader can access and run/test them.
* If you use outside resources, please refer them (in any formats). Include the links to the resources if necessary.

# Introduction

## Background

  Adverse drug-drug interaction (DDI) is the unintended molecular interactions between drugs. It’s a prominent cause of patient morbidities/mortalities, incurring higher costs and risking patient safety. The difficulty of mitigating this issue stems from a couple of factors: 

  * The molecular structure of drugs are complex, consisting of many units and substructures
  * Drug development is a process that requires highly specialized knowledge
  * Trials to test drugs and post-market surveillance are long and expensive processes

  With respect to applying ML to this topic, there are also a couple of issues:
  
  * There is a relatively light amount of training data that exists, due to the slow reporting of instances of DDIs
  * The results that models can give and extracting the reasoning for why a DDI is occurring can be hard to glean for researchers

  There is major interest in predicting whether two drugs will interact (especially during the design process) to reduce testing/development costs and improving patient safety.

### The Current State of the Art
 
  Deep learning models have been successfully used to predict DDIs

## CASTER
  The ChemicAl SubstrucTurE Representation framework, or CASTER, was introduced to predict DDIs, improving on the weaknesses of prior works.
  * what did the paper propose
  * what is the innovations of the method
  * how well the proposed method work (in its own metrics)
  * what is the contribution to the reasearch regime (referring the Background above, how important the paper is to the problem).


# Scope of Reproducibility:

The authors of CASTER presented the following hypotheses:

1.  The CASTER model will provide more accurate DDI predictions when compared with other established models.

2.  The usage of unlabelled data to generate frequent sub-structure features improves performance in situations with limited labeled datasets.

3.  CASTER’s sub-structure dictionary can help human operators better comprehend the final result.


# Methodology

The methodology at least contains two subsections **data** and **model** in your experiment.

In [6]:
#Just a bunch of imports

import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils import data
from torch import nn 
import copy

import raw_to_vecs
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import roc_auc_score, precision_recall_curve
from sklearn.model_selection import KFold
torch.manual_seed(2)    # reproducible torch:2 np:3
np.random.seed(3)



##  Data
The data that the model ingests consists of 3 datasets:
  * unsup_dataset.csv = A dataset of randomly combined pairs of SMILEs strings drawn from FooDB (a db of food constituent molecules) and all drugs drawn from DrugBank
  * BIOSNAP/sup*.csv = A dataset from Stanford's Biomedical Network Dataset indicating pairs of SMILEs strings and presence of DDI.
  * subword_units_map.csv = 1722 frequent patterns extracted from the above strings
  
Data includes raw data (MIMIC III tables), descriptive statistics (our homework questions), and data processing (feature engineering).
  * Source of the data: where the data is collected from; if data is synthetic or self-generated, explain how. If possible, please provide a link to the raw datasets.
  * Statistics: include basic descriptive statistics of the dataset like size, cross validation split, label distribution, etc.
  * Data process: how do you munipulate the data, e.g., change the class labels, split the dataset to train/valid/test, refining the dataset.
  * Illustration: printing results, plotting figures for illustration.
  * You can upload your raw dataset to Google Drive and mount this Colab to the same directory. If your raw dataset is too large, you can upload the processed dataset and have a code to load the processed dataset.

In [10]:

def load_raw_data(raw_data_dir):
  # implement this function to load raw data to dataframe/numpy array/tensor
  return None

# calculate statistics
def calculate_stats(raw_data):
  # implement this function to calculate the statistics
  # it is encouraged to print out the results
  return None

# process raw data
def process_data(raw_data):
  # implement this function to process the data as you need
  return None

PARAMS = {'batch_size': 256,
          'shuffle': True,
          'num_workers': 4}

#5-fold
df_unsup = pd.read_csv('data' + '/unsup_dataset.csv', names = ['idx', 'input1_SMILES', 'input2_SMILES', 'type']).drop(0)# pairs dataframe input1_smiles, input2_smiles
df_ddi = pd.read_csv('data/BIOSNAP/sup_train_val.csv')  # ddi dataframe drug1_smiles, drug2_smiles
kf = KFold(n_splits = 8, shuffle = True, random_state = 3)

#Get the 1st fold index
fold_index = next(kf.split(df_ddi), None)

ids_unsup = df_unsup.index.values
partition_sup = {'train': fold_index[0], 'val': fold_index[1]}
labels_sup = df_ddi.label.values

unsup_set = raw_to_vecs.unsup_data(ids_unsup, df_unsup)
unsup_generator = data.DataLoader(unsup_set, **PARAMS)

training_set = raw_to_vecs.sup_data(partition_sup['train'], labels_sup, df_ddi)
training_generator_sup = data.DataLoader(training_set, **PARAMS)

validation_set = raw_to_vecs.sup_data(partition_sup['val'], labels_sup, df_ddi)
validation_generator_sup = data.DataLoader(validation_set, **PARAMS)

print(len(df_unsup))
print(len(unsup_generator))
print(len(training_generator_sup))
print(len(validation_generator_sup))

''' 
you can load the processed data directly
processed_data_dir = '/content/gdrive/My Drive/Colab Notebooks/<path-to-raw-data>'
def load_processed_data(raw_data_dir):
  pass
'''

441853
1726
228
33


" \nyou can load the processed data directly\nprocessed_data_dir = '/content/gdrive/My Drive/Colab Notebooks/<path-to-raw-data>'\ndef load_processed_data(raw_data_dir):\n  pass\n"

##   Model

![sample_image.png](img/caster_block_diagram.png)

The model includes the model definitation which usually is a class, model training, and other necessary parts.
  * Model architecture: layer number/size/type, activation function, etc
  * Training objectives: loss function, optimizer, weight of each loss term, etc
  * Others: whether the model is pretrained, Monte Carlo simulation for uncertainty analysis, etc
  * The code of model should have classes of the model, functions of model training, model validation, etc.
  * If your model training is done outside of this notebook, please upload the trained model here and develop a function to load and test it.

In [ ]:
class my_model():
  # use this class to define your model
  pass

model = my_model()
loss_func = None
optimizer = None

def train_model_one_iter(model, loss_func, optimizer):
  pass

num_epoch = 10
# model training loop: it is better to print the training/validation losses during the training
for i in range(num_epoch):
  train_model_one_iter(model, loss_func, optimizer)
  train_loss, valid_loss = None, None
  print("Train Loss: %.2f, Validation Loss: %.2f" % (train_loss, valid_loss))


## Ablation: Logistic Regression

Replacement of the deep predictor of the functional representation with a logistic regression. The molecule’s functional representation (sub-structured/post pattern mining) is fed through the deep predictor. The logistic regression model, though far lighter in parameter count, was not chosen due its weaker performance.

In [12]:
class simple_log_reg(nn.Module):

    def __init__(self, n_inputs, n_outputs):
        super(simple_log_reg, self).__init__()
        self.linear = torch.nn.Linear(n_inputs, n_outputs)
    
    def forward(self,x):
        pred = torch.sigmoid(self.linear(x))
        return pred


# Results
In this section, you should finish training your model training or loading your trained model. That is a great experiment! You should share the results with others with necessary metrics and figures.

Please test and report results for all experiments that you run with:

*   specific numbers (accuracy, AUC, RMSE, etc)
*   figures (loss shrinkage, outputs from GAN, annotation or label of sample pictures, etc)


In [ ]:
# metrics to evaluate my model

# plot figures to better show the results

# it is better to save the numbers and figures for your presentation.

# Model comparison

# Discussion

In this section,you should discuss your work and make future plan. The discussion should address the following questions:
  * Make assessment that the paper is reproducible or not.
  * Explain why it is not reproducible if your results are kind negative.
  * Describe “What was easy” and “What was difficult” during the reproduction.
  * Make suggestions to the author or other reproducers on how to improve the reproducibility.
  * What will you do in next phase.



In [ ]:
# no code is required for this section
'''
if you want to use an image outside this notebook for explanaition,
you can read and plot it here like the Scope of Reproducibility
'''

# References

1.   Huang, K., Xiao, C., Hoang, T., Glass, L., & Sun, J. (2020, April). Caster: Predicting drug interactions with chemical substructure representation. In Proceedings of the AAAI conference on artificial intelligence (Vol. 34, No. 01, pp. 702-709).

